# RUL Prediction with LSTM
a. Xs' only
b. Xs' and $\theta$

1. Load flight effects model and validate performance
2. set up hyperparameter tuning for RUL prediction network (LSTM)
3. Evaluate different lookback windows or implement adaptive window

In [1]:
import os
import sys
import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

base_dir = os.path.dirname(os.getcwd())
print(base_dir)
sys.path.insert(1, base_dir)
from package.api import DB as api
import package.utils as utils
import package.tuning as tuning
utils.check_gpu()

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers, optimizers, metrics
#import tensorflow_addons as tfa

import keras_tuner as kt

from kerastuner_tensorboard_logger import (
    TensorBoardLogger,
    setup_tb  # Optional
)


%matplotlib inline
%load_ext autoreload
%autoreload 2

/home/macslab/phm2021_data_challenge
2.8.0
[INFO] GPU?: <True> [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


## 0. Connect to db and load the data

In [2]:
# THESE ARE YOUR CREDENTIALS IN PLAIN TEXT!
params = {'datasource.username': 'macslab', # the username of the logged in user
            'datasource.password': 'Ch0colate!', 
            'datasource.database': 'ncmapss_db', # <- NO CHANGE 
            'datasource.url': 'localhost', # <- or your database installation location
            'datasource.port': '5432'} # <- most likely don't change
#print(params)
db, cur =  api.connect(params)
db.set_session(autocommit=True)
del(params)

[INFO] connecting to db.
[INFO] connected.


In [3]:
units = api._get_units(db=db)
units.head()

,id,serial_number,age,eol,rul,group_id,Fc,unit,dataset
0,1,xhJzs6e3,0.0,100.0,100.0,1,1,1,DS01-005
1,2,a2wH8lJP,0.0,75.0,75.0,1,3,2,DS01-005
2,3,zwaWwxPl,0.0,100.0,100.0,1,2,3,DS01-005
3,4,ByMDLFeB,0.0,95.0,95.0,1,1,4,DS01-005
4,5,rcedq8Jy,0.0,89.0,89.0,1,3,5,DS01-005


In [ ]:
# import Queue
# from threading import Thread

# units = api._get_units(db=db)
# tables = ['summary_tb', 'telemetry_tb']

# def func(units, tables, downsample):
#     return str(units) + str(tables) + str(downsample)
# def delegate():
#     return func('units', ' tables', 20)
# units = api._get_units(db=db)
# data_frames = []
# for i in range(int(len(units)/10)):
    
    
    # df = api._get_data(db=db,
    #                    units=units,
    #                    tables=tables,
    #                    downsample=20).astype(np.float32)
    

In [5]:
train_df = pd.read_csv('train_df.csv', index_col=0)
train_pop = pd.read_csv('train_pop.csv', index_col=0)
train_y = pd.read_csv('train_y.csv', index_col=0)

In [4]:
tables = ['summary_tb', 'telemetry_tb']
downsample=20
df = api._get_data(db=db,
                   units=pd.unique(units.id),
                   hs=0,
                   tables=tables,
                   downsample=downsample).astype(np.float32)
utils.add_time_column(units=pd.unique(units.id), df=df)
utils.add_rul_column(units=pd.unique(units.id), df=df)

## 1. Make Xs_prime and y 

In [6]:
X_cols = ['Mach', 'alt', 'TRA', 'T2', 'time']## disregard comments #, 'Fc'] # will also have Fc_1, Fc_2, Fc_3
y_cols = ['Wf', 'Nf', 'Nc', 'T24', 'T30', 'T48', 'T50', 'P15', 'P2', 'P21', 'P24', 'Ps30', 'P40', 'P50']
a_cols = ['cycle', 'hs', 'Fc', 'asset_id']

model = keras.models.load_model('models/ncmapss/flight_effects.h5')
yscaler = joblib.load('models/ncmapss/flight_effects_yscaler.pkl')
xscaler = joblib.load('models/ncmapss/flight_effects_xscaler.pkl')

# Xs_prime = []
# idx = np.random.randint(len(y_cols))
# polys = []

# df[X_cols] = xscaler.transform(df[X_cols])
# pred = model.predict(df[X_cols])

# for asset_id in pd.unique(df.asset_id):
#     X_full = df[df.asset_id == asset_id][X_cols]
#     pred = model.predict(X_full)
#     # pred = model.predict(xscaler.transform(X_full))
#     Xs_prime.append(pred)
    
#    y_full = df[df.asset_id == asset_id][y_cols]
#    trace = yscaler.transform(y_full)
#     res = trace - pred

#     x = np.arange(0, len(res[:,0]))
#     x = x[0:len(x):10]
#     for i in range(0, len(y_cols)):
#         y = res[:,i]
#         y = y[0:len(y):10]
#         reg = np.poly1d(np.polyfit(x, y, 6))
#         polys.append(reg)

#     fig = plt.figure(figsize=(12,12))
#     filtered = np.zeros(res.shape)
#     x = np.arange(0, len(res[:,0]))
#     for j in range(len(y_cols)):
#         y = res[:,j]
#         p = polys[j]

#         diff = np.zeros((len(y),))
#         for i in range(len(y)):
#             predp = p(y[i])
#             diff[i] = y[i] - predp

#         mean = np.mean(diff)
#         std = np.std(diff)

#         filt = np.zeros((len(y),))
#         for i in range(len(res)):
#             if np.abs(np.abs(y[i]) - np.abs(p(x[i]))) > np.abs(mean)+3*std:
#                 filt[i] = p(x[i])
#             else:
#                 filt[i] = y[i]

#         filtered[:,j] = filt
#         utils.plot_trace_samples(fig, y, filt, p, j)
#     plt.show()
#     Xs_prime.append(pred)

2022-02-14 08:23:48.679124: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-02-14 08:23:48.951359: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10147 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060, pci bus id: 0000:01:00.0, compute capability: 8.6


### now we have Xs_prime and y

In [6]:
#xsp = np.concatenate(tuple(Xs_prime), axis=0)
dfx = pd.DataFrame(data=pred, columns=y_cols)
dfx = pd.concat([dfx, df[X_cols], df[a_cols]], axis=1)
dfx['rul'] = df['rul'].values
dfx.time = dfx.time + (dfx.cycle -1)

dfx.head()

,Wf,Nf,Nc,T24,T30,T48,T50,P15,P2,P21,...,Mach,alt,TRA,T2,time,cycle,hs,Fc,asset_id,rul
0,0.818128,0.841762,0.887926,0.903558,0.881721,0.868859,0.888709,0.924820,0.954334,0.924977,...,0.480052,0.004090,0.756065,0.955631,36.000000,37.0,0.0,1.0,1.0,64.0
1,0.783579,0.814023,0.866464,0.885147,0.859084,0.844705,0.867876,0.908484,0.947527,0.908656,...,0.487148,0.010459,0.721024,0.952158,36.004505,37.0,0.0,1.0,1.0,63.995495
2,0.777578,0.815973,0.864358,0.879604,0.856746,0.844110,0.865498,0.896730,0.933655,0.896904,...,0.486223,0.018576,0.726415,0.942436,36.009009,37.0,0.0,1.0,1.0,63.990991
3,0.777463,0.819259,0.865445,0.881605,0.857858,0.845077,0.864297,0.895452,0.931826,0.895701,...,0.498471,0.023665,0.730458,0.943244,36.013514,37.0,0.0,1.0,1.0,63.986486
4,0.774587,0.819201,0.864389,0.881093,0.856722,0.843864,0.862012,0.892926,0.929568,0.893229,...,0.505742,0.027411,0.730458,0.942958,36.018018,37.0,0.0,1.0,1.0,63.981982


## 2. split sets and view feature distributions

In [8]:
train_df, train_y, val_df, val_y, test_df, test_y = utils.train_test_split(df=dfx, units=list(pd.unique(units.id)), y_labels=['rul'], t_labels=a_cols + y_cols + X_cols, train_pct = .80, val_pct=.10, test_pct=.10, verbose=True)

90
72
9
9
train, val, test set counts: 72, 9, 9
90 90
train units: [33, 71, 13, 10, 68, 57, 74, 14, 84, 75, 24, 21, 73, 48, 35, 12, 9, 70, 69, 82, 43, 55, 22, 54, 4, 58, 88, 26, 45, 1, 86, 50, 63, 44, 72, 19, 77, 31, 90, 60, 3, 39, 8, 46, 15, 36, 51, 25, 18, 65, 5, 40, 81, 89, 61, 20, 17, 80, 53, 34, 41, 62, 78, 37, 64, 76, 59, 49, 28, 87, 16, 85]
val units: [7, 66, 6, 29, 27, 11, 38, 47, 32]
test units: [2, 56, 30, 52, 79, 67, 83, 23, 42]


In [11]:
train_y = pd.DataFrame(data=train_y, columns=['rul'])
train_y.head()

,rul
0,64.000000
1,63.995495
2,63.990990
3,63.986488
4,63.981983


In [10]:
train_pop = pd.concat([train_df.pop(x) for x in ['asset_id', 'cycle', 'hs', 'Fc']], axis=1)
val_pop = pd.concat([val_df.pop(x) for x in ['asset_id', 'cycle', 'hs', 'Fc']], axis=1)
test_pop = pd.concat([test_df.pop(x) for x in ['asset_id', 'cycle', 'hs', 'Fc']], axis=1)

In [12]:
train_df.to_csv('train_df.csv')
train_pop.to_csv('train_pop.csv')
train_y.to_csv('train_y.csv')

In [ ]:
utils.plot_feature_distributions(df=train_df, feature_range=(0,1), figsize=(12,4))
utils.plot_feature_distributions(df=val_df, feature_range=(0,1), figsize=(12,4))
utils.plot_feature_distributions(df=test_df, feature_range=(0,1), figsize=(12,4))

### plot feature importances

In [ ]:
from xgboost import XGBRegressor
importance_model = XGBRegressor()
importance_model.fit(train_df.values, train_y)

scoring = ['explained_variance',
           'max_error']#,
           #'neg_mean_absolute_error',
           #'neg_root_mean_squared_error',
           #'r2']

utils.plot_feature_importances(model=importance_model,
                               features=train_df.values,
                               feature_labels=y_cols + X_cols,
                               target=train_y,
                               target_label='rul',
                               scoring=scoring)

utils.plot_feature_importance(features=train_df.values,
                              feature_labels=y_cols + X_cols,
                              target=train_y,
                              target_label='rul',
                              figsize=(9,4))

In [9]:
important_features = list(train_df.columns) #['Wf','Nc', 'T24', 'T48', 'T50', 'P2', 'Ps30', 'P40']
important_features

['Wf',
 'Nf',
 'Nc',
 'T24',
 'T30',
 'T48',
 'T50',
 'P15',
 'P2',
 'P21',
 'P24',
 'Ps30',
 'P40',
 'P50',
 'Mach',
 'alt',
 'TRA',
 'T2',
 'time']

## 2. temporalize the data

In [10]:
lookback = 100
horizon = 1
n_out = 1
n_features = train_df[important_features].shape[1]

X_train, y_train = utils.temporalize_data(train_df[important_features].values, train_y.values, lookback, horizon, n_features, n_out)
#train_pop_temp, y_pop = utils.temporalize_data(train_pop.values, train_y, lookback, horizon, len(train_pop.columns), n_out)

# X_val, y_val = utils.temporalize_data(val_df.values, val_y, lookback, horizon, n_features, n_out)
# val_pop_temp, y_pop = utils.temporalize_data(val_pop.values, val_y, lookback, horizon, n_features, n_out)

# X_test, y_test = utils.temporalize_data(test_df.values, test_y, lookback, horizon, n_features, n_out)
# test_pop_temp, y_pop = utils.temporalize_data(test_pop.values, test_y, lookback, horizon, n_features, n_out)

KeyboardInterrupt: 

# TIM check gpu memory first

In [7]:

input_shape = (lookback, n_features)
my_tuning = tuning.Tuning(input_shape, n_out)
bayesian_tuning = my_tuning.bayesian_search(objective='root_mean_squared_error',
                                            mode='min',
                                            max_trials=256,
                                            alpha=.00025,
                                            beta=2.75,
                                            epochs=5,
                                            executions_per_trial=1,
                                            hypermodel=my_tuning.create_lstm_hypermodel,
                                            directory='logs/ncmapss2/bayesiansearch',
                                            project_name='lstm',
                                            logger=TensorBoardLogger(
                                                metrics=['root_mean_squared_error'],
                                                         logdir='logs/ncmapss2/hparams'
                                            ),
                                            X=X_train,
                                            y=y_train)

bayesian_tuning_params = bayesian_tuning.get_best_hyperparameters(num_trials=1)[0]
bayesian_tuning_model = bayesian_tuning.get_best_models()[0]
print(bayesian_tuning_params.values)
bayesian_tuning_model.summary()

NameError: name 'lookback' is not defined